### Set up the environment variables for Pyspark, Java, Spark, and python
- 오류 발생 시
- 이미 환경변수 설정 했기 때문에 실행 x

In [ ]:
import os
import sys
os.environ['JAVA_HOME'] = "C:\Java"
os.environ['SPARK_HOME'] = "C:\spark-3.2.3"
os.environ['PYLIB'] = "C:\spark-3.2.3\python\lib"
sys.path.insert(0,os.environ['PYLIB']+"\py4j-0.10.9.5-src.zip")
sys.path.insert(0,os.environ['PYLIB']+"\pyspark.zip")

#### Employee.csv
- 한글포함
- ANSI 형식으로 encoding

In [2]:
import pyspark
import pandas as pd

In [3]:
pandasDF = pd.read_csv("Employee.csv",encoding="ansi")
pandasDF

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,1,남성,15,경영자,57000.0,27000,98,144,No
1,2,남성,16,사무직,40200.0,18750,98,36,No
2,3,여성,12,사무직,21450.0,12000,98,381,No
3,4,여성,8,사무직,21900.0,13200,98,190,No
4,5,남성,15,사무직,45000.0,21000,98,138,No
...,...,...,...,...,...,...,...,...,...
469,470,남성,12,사무직,26250.0,15750,64,69,Yes
470,471,남성,15,사무직,26400.0,15750,64,32,Yes
471,472,남성,15,사무직,39150.0,15750,63,46,No
472,473,여성,12,사무직,21450.0,12750,63,139,No


spark data 만들기

In [4]:
from pyspark.sql import SparkSession

In [5]:
스파크 = SparkSession.builder.appName('Test').getOrCreate()

In [6]:
스파크

### 웹브라우저에서 localhost:4040 연결

### Pyspark에서 hdfs 데이터 불러오기
Spark 경로를 찾지 못하는 경우
```Python
!pip install findspark
import findspark
import os
findspark.find()
findspark.init(os.environ.get("SPARK_HOME"))
```

In [7]:
sparkDF = 스파크.read.csv("Employee.csv")
sparkDF.show()

+---+------+----+------+------+--------+-------+-------+--------+
|_c0|   _c1| _c2|   _c3|   _c4|     _c5|    _c6|    _c7|     _c8|
+---+------+----+------+------+--------+-------+-------+--------+
| id|gender|educ|jobcat|salary|salbegin|jobtime|prevexp|minority|
|  1|  ����|  15| �濵��| 57000|   27000|     98|    144|      No|
|  2|  ����|  16| �繫��| 40200|   18750|     98|     36|      No|
|  3|  ����|  12| �繫��| 21450|   12000|     98|    381|      No|
|  4|  ����|   8| �繫��| 21900|   13200|     98|    190|      No|
|  5|  ����|  15| �繫��| 45000|   21000|     98|    138|      No|
|  6|  ����|  15| �繫��| 32100|   13500|     98|     67|      No|
|  7|  ����|  15| �繫��| 36000|   18750|     98|    114|      No|
|  8|  ����|  12| �繫��| 21900|    9750|     98|      0|      No|
|  9|  ����|  15| �繫��| 27900|   12750|     98|    115|      No|
| 10|  ����|  12| �繫��| 24000|   13500|     98|    244|      No|
| 11|  ����|  16| �繫��| 30300|   16500|     98|    143|      No|
| 12|  ����|   8| �繫�

# 컴퓨터 환경에 따른 실행 결과 차이

CMD에서 start-dfs.cmd, start-yarn.cmd로 hadoop을 실행한 후 `hadoop fs -put Employee.csv spark_Employee.csv`가 바로 안되는 경우

- put: `spark_Employee.csv': No such file or directory: `hdfs://localhost:9000/user/SM-PC/spark_Employee.csv'
- 위와 같은 오류로 put이 안됨.

In [9]:
# Hadoop 실행 후(stat-all.cmd)
## CMD> hadoop fs -mkdir /Spark
## CMD > hadoop fs -put Employee.csv /Spark/spark_Employee.csv

sparkDF = 스파크.read.csv("hdfs://localhost:9000/Spark/spark_Employee.csv")
sparkDF.show()

### 한글, 변수명(id)를 셀로 가져오는 문제

+---+------+----+------+------+--------+-------+-------+--------+
|_c0|   _c1| _c2|   _c3|   _c4|     _c5|    _c6|    _c7|     _c8|
+---+------+----+------+------+--------+-------+-------+--------+
| id|gender|educ|jobcat|salary|salbegin|jobtime|prevexp|minority|
|  1|  ����|  15| �濵��| 57000|   27000|     98|    144|      No|
|  2|  ����|  16| �繫��| 40200|   18750|     98|     36|      No|
|  3|  ����|  12| �繫��| 21450|   12000|     98|    381|      No|
|  4|  ����|   8| �繫��| 21900|   13200|     98|    190|      No|
|  5|  ����|  15| �繫��| 45000|   21000|     98|    138|      No|
|  6|  ����|  15| �繫��| 32100|   13500|     98|     67|      No|
|  7|  ����|  15| �繫��| 36000|   18750|     98|    114|      No|
|  8|  ����|  12| �繫��| 21900|    9750|     98|      0|      No|
|  9|  ����|  15| �繫��| 27900|   12750|     98|    115|      No|
| 10|  ����|  12| �繫��| 24000|   13500|     98|    244|      No|
| 11|  ����|  16| �繫��| 30300|   16500|     98|    143|      No|
| 12|  ����|   8| �繫�

In [10]:
# 한글 깨짐 해결
## cp949가 어떤 것의 alias?
sparkDF = 스파크.read.option('encoding','cp949').csv("hdfs://localhost:9000/Spark/spark_Employee.csv")
sparkDF # 형태를 보여줌

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string]

In [11]:
sparkDF.show()

+---+------+----+------+------+--------+-------+-------+--------+
|_c0|   _c1| _c2|   _c3|   _c4|     _c5|    _c6|    _c7|     _c8|
+---+------+----+------+------+--------+-------+-------+--------+
| id|gender|educ|jobcat|salary|salbegin|jobtime|prevexp|minority|
|  1|  남성|  15|경영자| 57000|   27000|     98|    144|      No|
|  2|  남성|  16|사무직| 40200|   18750|     98|     36|      No|
|  3|  여성|  12|사무직| 21450|   12000|     98|    381|      No|
|  4|  여성|   8|사무직| 21900|   13200|     98|    190|      No|
|  5|  남성|  15|사무직| 45000|   21000|     98|    138|      No|
|  6|  남성|  15|사무직| 32100|   13500|     98|     67|      No|
|  7|  남성|  15|사무직| 36000|   18750|     98|    114|      No|
|  8|  여성|  12|사무직| 21900|    9750|     98|      0|      No|
|  9|  여성|  15|사무직| 27900|   12750|     98|    115|      No|
| 10|  여성|  12|사무직| 24000|   13500|     98|    244|      No|
| 11|  여성|  16|사무직| 30300|   16500|     98|    143|      No|
| 12|  남성|   8|사무직| 28350|   12000|     98|     26|     Yes|
| 13

In [12]:
# 헤더 지정
sparkDF = 스파크.read.option('encoding','cp949').option('header','true').csv("hdfs://localhost:9000/Spark/spark_Employee.csv")

In [13]:
sparkDF.show(5)  # 데이터를 보고싶을 때 spark dataframe에서 .show()

+---+------+----+------+------+--------+-------+-------+--------+
| id|gender|educ|jobcat|salary|salbegin|jobtime|prevexp|minority|
+---+------+----+------+------+--------+-------+-------+--------+
|  1|  남성|  15|경영자| 57000|   27000|     98|    144|      No|
|  2|  남성|  16|사무직| 40200|   18750|     98|     36|      No|
|  3|  여성|  12|사무직| 21450|   12000|     98|    381|      No|
|  4|  여성|   8|사무직| 21900|   13200|     98|    190|      No|
|  5|  남성|  15|사무직| 45000|   21000|     98|    138|      No|
+---+------+----+------+------+--------+-------+-------+--------+
only showing top 5 rows



In [14]:
type(sparkDF)  # pyspark가 제공하는 데이터 프레임

pyspark.sql.dataframe.DataFrame

In [15]:
sparkDF.head(5)

[Row(id='1', gender='남성', educ='15', jobcat='경영자', salary='57000', salbegin='27000', jobtime='98', prevexp='144', minority='No'),
 Row(id='2', gender='남성', educ='16', jobcat='사무직', salary='40200', salbegin='18750', jobtime='98', prevexp='36', minority='No'),
 Row(id='3', gender='여성', educ='12', jobcat='사무직', salary='21450', salbegin='12000', jobtime='98', prevexp='381', minority='No'),
 Row(id='4', gender='여성', educ='8', jobcat='사무직', salary='21900', salbegin='13200', jobtime='98', prevexp='190', minority='No'),
 Row(id='5', gender='남성', educ='15', jobcat='사무직', salary='45000', salbegin='21000', jobtime='98', prevexp='138', minority='No')]

In [16]:
# 데이터의 구조
sparkDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- educ: string (nullable = true)
 |-- jobcat: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- salbegin: string (nullable = true)
 |-- jobtime: string (nullable = true)
 |-- prevexp: string (nullable = true)
 |-- minority: string (nullable = true)



In [17]:
# 위의 과정과 같은 걸 한 번에
## smu: user name
data = 스파크.read.csv("hdfs://localhost:9000/Spark/spark_Employee.csv", header=True, encoding="cp949", inferSchema="true")
data.show()

+---+------+----+------+--------+--------+-------+-------+--------+
| id|gender|educ|jobcat|  salary|salbegin|jobtime|prevexp|minority|
+---+------+----+------+--------+--------+-------+-------+--------+
|  1|  남성|  15|경영자| 57000.0|   27000|     98|    144|      No|
|  2|  남성|  16|사무직| 40200.0|   18750|     98|     36|      No|
|  3|  여성|  12|사무직| 21450.0|   12000|     98|    381|      No|
|  4|  여성|   8|사무직| 21900.0|   13200|     98|    190|      No|
|  5|  남성|  15|사무직| 45000.0|   21000|     98|    138|      No|
|  6|  남성|  15|사무직| 32100.0|   13500|     98|     67|      No|
|  7|  남성|  15|사무직| 36000.0|   18750|     98|    114|      No|
|  8|  여성|  12|사무직| 21900.0|    9750|     98|      0|      No|
|  9|  여성|  15|사무직| 27900.0|   12750|     98|    115|      No|
| 10|  여성|  12|사무직| 24000.0|   13500|     98|    244|      No|
| 11|  여성|  16|사무직| 30300.0|   16500|     98|    143|      No|
| 12|  남성|   8|사무직| 28350.0|   12000|     98|     26|     Yes|
| 13|  남성|  15|사무직| 27750.0|   14250|   

### DataFrame을 다른 형식으로 변환하고 저장/불러오기
- pandas DataFrame $\Longleftrightarrow$ spark DataFrame
- csv $\Longleftrightarrow$ parquet

In [18]:
pandasDF_spark = sparkDF.toPandas()
pandasDF_spark.head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,1,남성,15,경영자,57000,27000,98,144,No
1,2,남성,16,사무직,40200,18750,98,36,No
2,3,여성,12,사무직,21450,12000,98,381,No
3,4,여성,8,사무직,21900,13200,98,190,No
4,5,남성,15,사무직,45000,21000,98,138,No


#### pandas DataFrame을 spark DataFrame으로
Apache Spark uses Apache Arrow which is an in-memory columnar format to transfer the data between Python and JVM. 
You need to enable to use Arrow as this is disabled by default and have Apache Arrow (PyArrow) install on all Spark cluster nodes using pip install pyspark[sql] or by directly downloading from Apache Arrow for Python.

In [19]:
### !pip install pyarrow
## 메모리 관리에 관한?
스파크.conf.set("spark.sql.execution.arrow.enabled","true")

### When an error occurs, 
Spark automatically fallback to non-Arrow optimization implementation, this can be controlled by spark.sql.execution.arrow.pyspark.fallback.enabled.
```pythpn
spark.conf.set("spark.sql.execution.arrow.pyspark.fallback.enabled","true")
```

In [20]:
sparkDF_pd  = 스파크.createDataFrame(pandasDF)  # 스파크DF <- 판다스DF 변환
# 빨간색이 뜨지만 워닝이지 에러는 아님. 한 번 더 실행하면 안뜸.

C:\Users\SM-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\sql\pandas\conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [21]:
type(sparkDF_pd)

pyspark.sql.dataframe.DataFrame

In [22]:
sparkDF_pd.printSchema()

root
 |-- id: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- educ: long (nullable = true)
 |-- jobcat: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- salbegin: long (nullable = true)
 |-- jobtime: long (nullable = true)
 |-- prevexp: long (nullable = true)
 |-- minority: string (nullable = true)



In [23]:
sparkDF_pd.show()

+---+------+----+------+--------+--------+-------+-------+--------+
| id|gender|educ|jobcat|  salary|salbegin|jobtime|prevexp|minority|
+---+------+----+------+--------+--------+-------+-------+--------+
|  1|  남성|  15|경영자| 57000.0|   27000|     98|    144|      No|
|  2|  남성|  16|사무직| 40200.0|   18750|     98|     36|      No|
|  3|  여성|  12|사무직| 21450.0|   12000|     98|    381|      No|
|  4|  여성|   8|사무직| 21900.0|   13200|     98|    190|      No|
|  5|  남성|  15|사무직| 45000.0|   21000|     98|    138|      No|
|  6|  남성|  15|사무직| 32100.0|   13500|     98|     67|      No|
|  7|  남성|  15|사무직| 36000.0|   18750|     98|    114|      No|
|  8|  여성|  12|사무직| 21900.0|    9750|     98|      0|      No|
|  9|  여성|  15|사무직| 27900.0|   12750|     98|    115|      No|
| 10|  여성|  12|사무직| 24000.0|   13500|     98|    244|      No|
| 11|  여성|  16|사무직| 30300.0|   16500|     98|    143|      No|
| 12|  남성|   8|사무직| 28350.0|   12000|     98|     26|     Yes|
| 13|  남성|  15|사무직| 27750.0|   14250|   

#### Pandas에서
작업하고 난 데이터를 저장하기.

In [24]:
pandasDF.to_csv('pandas_Employee.csv')
pandasDF.to_parquet('pandas_Employee.parquet')
pandasPQ  = pd.read_parquet('pandas_Employee.parquet')
pandasPQ.head()
# 내 컴퓨터의 D:/Bigdata/Examples 에 저장된다.

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,1,남성,15,경영자,57000.0,27000,98,144,No
1,2,남성,16,사무직,40200.0,18750,98,36,No
2,3,여성,12,사무직,21450.0,12000,98,381,No
3,4,여성,8,사무직,21900.0,13200,98,190,No
4,5,남성,15,사무직,45000.0,21000,98,138,No


In [29]:
pandasCSV = pd.read_csv('pandas_Employee.csv')
pandasCSV.head()

,Unnamed: 0,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,0,1,남성,15,경영자,57000.0,27000,98,144,No
1,1,2,남성,16,사무직,40200.0,18750,98,36,No
2,2,3,여성,12,사무직,21450.0,12000,98,381,No
3,3,4,여성,8,사무직,21900.0,13200,98,190,No
4,4,5,남성,15,사무직,45000.0,21000,98,138,No


#### Spark에서 
- sparkDF.write.csv("경로/파일이름")
- sparkDF.format('csv').save("경로/파일이름")
- 기존 파일이 있는 경우 
    - 덮어쓰기: sparkDF.write.mode('overwrite').csv("경로/파일이름")
    - 추가하기: sparkDF.write.mode('append').csv("경로/파일이름")
    - 무시하기: sparkDF.write.mode('ignore').csv("경로/파일이름")
    - 오류발생: sparkDF.write.mode('error').csv("경로/파일이름")  $\Leftarrow$ default
    

In [25]:
sparkDF.write.csv("hdfs://localhost:9000/Spark/hadoop_Employee.csv")
sparkDF.write.parquet("hdfs://localhost:9000/Spark/hadoop_Employee.parquet")
sparkPQ = 스파크.read.parquet("hdfs://localhost:9000/Spark/hadoop_Employee.parquet")
sparkPQ.show(5)
# hadoop에 저장된다.
## parquet를 불러올 땐 header, encoding 안해도 된다
## 정보가 다 저장되어 있음

+---+------+----+------+------+--------+-------+-------+--------+
| id|gender|educ|jobcat|salary|salbegin|jobtime|prevexp|minority|
+---+------+----+------+------+--------+-------+-------+--------+
|  1|  남성|  15|경영자| 57000|   27000|     98|    144|      No|
|  2|  남성|  16|사무직| 40200|   18750|     98|     36|      No|
|  3|  여성|  12|사무직| 21450|   12000|     98|    381|      No|
|  4|  여성|   8|사무직| 21900|   13200|     98|    190|      No|
|  5|  남성|  15|사무직| 45000|   21000|     98|    138|      No|
+---+------+----+------+------+--------+-------+-------+--------+
only showing top 5 rows



In [28]:
#sparkDF_pd.write.csv("/Spark/sparkdf_Employee.csv")
sparkDF_pd = 스파크.read.csv("hdfs://localhost:9000/Spark/hadoop_Employee.csv",header=True)
sparkDF_pd.show(5)
# header가 사라졌다.

+---+----+---+------+-----+-----+---+---+---+
|  1|남성| 15|경영자|57000|27000| 98|144| No|
+---+----+---+------+-----+-----+---+---+---+
|  2|남성| 16|사무직|40200|18750| 98| 36| No|
|  3|여성| 12|사무직|21450|12000| 98|381| No|
|  4|여성|  8|사무직|21900|13200| 98|190| No|
|  5|남성| 15|사무직|45000|21000| 98|138| No|
|  6|남성| 15|사무직|32100|13500| 98| 67| No|
+---+----+---+------+-----+-----+---+---+---+
only showing top 5 rows



In [33]:
import pyspark
from pyspark.sql import SparkSession

스파크 = SparkSession.builder.appName("hdfs").getOrCreate()

In [34]:
## CMD> hadoop fs -mkdir /Text
## CMD> hadoop fs -put 제주호텔리뷰.csv /Text/.
data = 스파크.read.csv("hdfs://localhost:9000/Text/jeju_hotel.csv", header=True, inferSchema="true")
data.show()

+------+-----------------------------------+
|rating|                               text|
+------+-----------------------------------+
|     4|  여행에 집중할수 있게 편안한 휴...|
|     4|     2일 이상 연박시 침대, 이불,...|
|     4|  지인에소개로온 호텔  깨끗하고 ...|
|     5|  방에 딱 들어서자마자 눈이 휘둥...|
|     5|  저녁에 맥주한잔 하는게 좋아서 ...|
|     1|바다전망이라해서 기대했는데 영아...|
|     5|  손님이 없는 날이라고 가장 바다...|
|     5| 엄마와 둘이 여행왔다가 가격대비...|
|     5|  딸 둘과의 4일동안의 제주여행줌...|
|     5|     제주여행 2일차!!! 호텔 휘슬...|
|     5|   예전에 그랜드 호텔일 때 저희 ...|
|     5| 지금까지 제주여행을 다니면서 여...|
|     5|  엄마랑 첫 제주도 여행인데 침구...|
|     5| 친구가 제주도에 놀러와서 투숙을...|
|     5| 차를 좋아하는 아이들에게는 최고...|
|     5| 직원분들은 눈만 마주쳐도 도와주...|
|     5|  조식조타하여 일부러 저녁 많이 ...|
|     5|  신제주에 위치한 매우 깨끗한 호...|
|     4|    모녀 여행 중 제주시 1박을 위...|
|     5| 슈페리어킹룸에 하루 숙박한 후기...|
+------+-----------------------------------+
only showing top 20 rows



In [35]:
pd_df = data.toPandas()

In [36]:
pd_df.head()

,rating,text
0,4,여행에 집중할수 있게 편안한 휴식을 제공하는 호텔이었습니다. 위치선정 또한 적당한 ...
1,4,"2일 이상 연박시 침대, 이불, 베게등 침구류 교체 및 어메니티 보강이 필요해 보입..."
2,4,지인에소개로온 호텔 깨끗하고 좋은거같아요 처음에는 없는게 많아 많이 당황했는데 ...
3,5,방에 딱 들어서자마자 눈이 휘둥그레질정도로 이렇게 넓은 호텔 처음 와본 것 같아요!...
4,5,저녁에 맥주한잔 하는게 좋아서 렌트 안하고 뚜벅이 하기로 했는데 호텔 바로 앞에 버...


In [37]:
# 폴더 생성하며 해당 폴더에 지정한 형태로 파일 저장
## 헤더 넣고 세이브
## Text 안의 csv라는 폴더에
data.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs://localhost:9000/Text/csv")

In [38]:
data.write.parquet("hdfs://localhost:9000/Text/parquet")

In [41]:
sparkDF_jeju = 스파크.read.csv("hdfs://localhost:9000/Text/csv",header=True)
sparkDF_jeju.show(5)

+------+---------------------------------+
|rating|                             text|
+------+---------------------------------+
|     4|여행에 집중할수 있게 편안한 휴...|
|     4|   2일 이상 연박시 침대, 이불,...|
|     4|지인에소개로온 호텔  깨끗하고 ...|
|     5|방에 딱 들어서자마자 눈이 휘둥...|
|     5|저녁에 맥주한잔 하는게 좋아서 ...|
+------+---------------------------------+
only showing top 5 rows

